In [103]:
import pandas as pd
import numpy as np
# Option to display numpy arrays without truncating them
np.set_printoptions(threshold=np.inf)

In [9]:
# List of states - use to import data
states=["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

In [82]:
# State CSV files named with underscores - modify state list
new_states=[]

for state in states:
    new_states.append(state.upper().replace(' ','_'))

In [ ]:
# Download state respiratory disease files from the following URL
# http://ghdx.healthdata.org/record/ihme-data/united-states-chronic-respiratory-disease-mortality-rates-county-1980-2014

In [209]:
# Loop through each filename, import, & append
base = "state_data/IHME_USA_COUNTY_RESP_DISEASE_MORTALITY_1980_2014_"
end = "_Y2017M09D26.CSV"

# Initialize dataframe with national dataset
start_df=pd.read_csv(base+'UNITED_STATES'+end)
start_df['state']='National'

# Now loop through each state
for state in new_states:
    
    before = 'Before length: '+str(len(start_df))
    
    path = base+state+end
    data = pd.read_csv(path)
    data['state']=state.title().replace('_'," ")
    
    start_df = start_df.append(data)
    
    after = 'After length: '+str(len(start_df))
    print(before + ' ' + after)    

ValueError: Integer column has NA values in column 4

In [229]:
# Currently just interested in asthma data
start_df = start_df[start_df['cause_name']=='Asthma']

In [ ]:
# Download smoking rate file from the following URL
# http://ghdx.healthdata.org/record/ihme-data/united-states-smoking-prevalence-county-1996-2012
smoking_df = pd.read_csv('smoking data/IHME_US_COUNTY_TOTAL_AND_DAILY_SMOKING_PREVALENCE_1996_2012.csv')

In [230]:
# Changes Males & Females to Male & Female so as to join more easily
smoking_df=smoking_df.replace('Males', 'Male').replace('Females','Female')
smoking_df.tail()

,state,county,sex,year,total_mean,total_lb,total_ub,daily_mean,daily_lb,daily_ub
162124,Wyoming,Weston County,Female,2011,23.66,18.60,29.36,18.27,14.35,23.29
162125,Wyoming,Weston County,Both,2011,26.06,21.28,31.36,19.95,16.14,24.70
162126,Wyoming,Weston County,Male,2012,27.32,22.98,32.09,20.31,16.79,24.49
162127,Wyoming,Weston County,Female,2012,22.71,17.95,28.11,17.18,13.55,21.77
162128,Wyoming,Weston County,Both,2012,24.97,20.40,29.99,18.72,15.16,23.10


In [160]:
# Some commands to quality check the datasets
len(smoking_df['year'].unique())
len(start_df['year_id'].unique())
print(start_df["sex"].unique())
print(smoking_df['sex'].unique())
start_df[start_df['state']=='Hawaii']
start_df[start_df['state']=='Delaware']['location_name'].unique()
start_df[start_df['state']=='Delaware']
smoking_df[smoking_df['state']=='Hawaii']
smoking_df[smoking_df['state']=='Delaware']['county'].unique()
smoking_df[smoking_df['state']=='Delaware']

In [254]:
# Merge two datasets
merged_df = pd.merge(start_df,smoking_df,how='left',left_on = ['location_name','state','sex','year_id'],
                     right_on = ['county','state','sex','year'])

In [255]:
# Remove unnecessary columns
merged_df=merged_df[['FIPS','state','county','year','sex','mx','lower','upper','total_mean','total_lb','total_ub','daily_mean','daily_lb',' daily_ub']]
# Rename columns to be more descriptive
merged_df=merged_df.rename(index=str, columns={"mx": "asthma_deaths_per_100k"
                                              ,"lower": "asthma_lb"
                                              ,"upper":"asthma_ub"
                                              ,'total_mean':"current_smoking_pct"
                                              ,'total_lb':"current_smoking_lb"
                                              ,'total_ub':"current_smoking_ub"
                                              ,'daily_mean':'daily_smoker_pct'
                                              ,'daily_lb':'daily_smoker_lb'
                                              ,' daily_ub':'daily_smoker_ub'
                                              ,'FIPS':'fips'})


In [276]:
# Drop na's
smoking_data = merged_df.dropna()
non_smoking_data = start_df.dropna()

In [277]:
# Convert fips to string; ignore warnings
smoking_data['fips'] = smoking_data['fips'].astype(int).astype(str)

/Users/edshafron/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [193]:
# Export data to CSV
# non_smoking_data.to_csv('non_smoking_data.csv')
smoking_data.to_csv('smoking_data.csv',index=False)